In [0]:

from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, initcap, when, lit, year, month, dayofmonth, concat, concat_ws
import time

In [0]:
catalog_name = "retail_dev"
schema_bronze = "bronze"
schema_silver = "silver"
schema_gold = "gold"
schema_auditoria ="auditoria"

In [0]:
sql_gold = f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_gold}
"""

spark.sql(sql_gold)

In [0]:
gold_dim_cliente = (
    spark.table(f"{catalog_name}.{schema_silver}.clientes")
    .select("id_cliente", "nombre", "email", "ciudad", "fecha_registro")
    .withColumn("creation_date", F.current_timestamp())
    .dropDuplicates(["id_cliente"])
)


In [0]:
display(gold_dim_cliente)

In [0]:
# Crear tabla destino
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_gold}.dim_clientes (
  id_cliente BIGINT,
  nombre STRING,
  email STRING,
  ciudad STRING,
  fecha_registro DATE,
  creation_date TIMESTAMP
) USING DELTA
""")

In [0]:
gold_dim_cliente.createOrReplaceTempView("gold_dim_cliente")
merge_start = time.time()
spark.sql(f"""
MERGE INTO {catalog_name}.{schema_gold}.dim_clientes tgt
USING gold_dim_cliente src
ON tgt.id_cliente = src.id_cliente
WHEN MATCHED AND src.creation_date > tgt.creation_date THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")
merge_end = time.time()

In [0]:
spark.table(f"{catalog_name}.{schema_gold}.dim_clientes").show(10)

In [0]:
import time
job_id = int(time.time())           # ID único basado en timestamp
job_run_id = int(time.time() * 100) # más granular
task_run_id = 1                     # o secuencial si prefieres


In [0]:
rows_in = gold_dim_cliente.count()
start = time.time()
spark.sql(f"""
INSERT INTO {catalog_name}.{schema_auditoria}.ingestion_log VALUES (
  {job_id},{job_run_id},{task_run_id},
  current_timestamp(), current_timestamp(),
  {int(time.time()-start)},
  'SUCCESS','dim_clientes','gold',
  {rows_in},{rows_in},0,0,0,{int(merge_end-merge_start)},current_timestamp()
)
""")

In [0]:
display(spark.table(f"{catalog_name}.{schema_auditoria}.ingestion_log"))


In [0]:

sql_command = f"""
  OPTIMIZE {catalog_name}.{schema_gold}.dim_clientes
  ZORDER BY (id_cliente)
"""

In [0]:
spark.sql(sql_command)


In [0]:
spark.sql(f"VACUUM {catalog_name}.{schema_gold}.dim_clientes")